In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# !pip install missingno
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda x: '%.3f' % x)
pd.set_option("display.width", 500)

In [2]:
def load_application_train():
    data = pd.read_csv("datasets/application_train.csv")
    return data

In [3]:
def load():
    data = pd.read_csv("datasets/titanic.csv")
    return data

In [4]:
def catch_outliers(dataframe, num_col, q1=0.25, q3=0.75):
    quartile1 = dataframe[num_col].quantile(q1)
    quartile3 = dataframe[num_col].quantile(q3)
    iqr = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * iqr
    low_limit = quartile1 - 1.5 * iqr

    # outliers = [dataframe[(dataframe[num_col] < low) | (dataframe[num_col] > up)]]
    return low_limit, up_limit


def check_outlier(dataframe, num_col):
    low_limit, up_limit = catch_outliers(dataframe, num_col)

    if dataframe[(dataframe[num_col] < low_limit) | (dataframe[num_col] > up_limit)].any(axis=None):
        return True
    else:
        return False

    
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """
    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.

    Parameters
    ----------
    dataframe: dataframe
        Değişken isimleri alınmak istenen dataframe'dir.
    cat_th: int, float
        Numerik fakat kategorik olan değişkenler için sınıf eşik değeri.
    car_th: int, float
        Kategorik fakat kardinal değişkenler için sınıf eşik değeri.

    Returns
    -------
    cat_cols: list
        Kategorik değişken listesi.
    num_cols: list
        Numerik değişken listesi.
    cat_but_car: list
        Kategorik görünümlü kardinal değişken listesi.

    Notes
    ------
    cat_cols + num_cols + cat_but_car = toplam değişken sayısı
    num_but_cat, cat_cols'un içerisinde.

    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if str(dataframe[col].dtypes) in ["object", "category", "bool"]]

    num_but_cats = [col for col in dataframe.columns if dataframe[col].dtypes in ["int64", "float64"] and
                    dataframe[col].nunique() < cat_th]

    cat_but_cars = [col for col in dataframe.columns if str(dataframe[col].dtypes) in ["object", "category"] and
                    dataframe[col].nunique() > car_th]

    cat_cols = cat_cols + num_but_cats

    cat_cols = [col for col in cat_cols if col not in cat_but_cars]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes in ["int64", "float64"]]
    num_cols = [col for col in num_cols if col not in num_but_cats]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f"cat_cols: {len(cat_cols)}")
    print(f"num_cols: {len(num_cols)}")
    print(f"cat_but_car: {len(cat_but_cars)}")
    print(f"num_but_cat: {len(num_but_cats)}")

    return cat_cols, num_cols, cat_but_cars


def reach_outliers(dataframe, num_col, index=False):
    up_limit, low_limit = catch_outliers(dataframe, num_col)

    if len(dataframe[(dataframe[num_col] < low_limit) | (dataframe[num_col] > up_limit)]) > 10:
        print(dataframe[(dataframe[num_col] < low_limit) | (dataframe[num_col] > up_limit)].head())
    else:
        print(dataframe[(dataframe[num_col] < low_limit) | (dataframe[num_col] > up_limit)])

    if index:
        return dataframe[(dataframe[num_col] < low_limit) | (dataframe[num_col] > up_limit)].index


def remove_outliers(dataframe, num_col):
    up_limit, low_limit = catch_outliers(dataframe, num_col)
    df_without_outliers = dataframe[~((dataframe[num_col] < low_limit) | (dataframe[num_col] > up_limit))]

    return df_without_outliers


def replace_with_thresholds(dataframe, num_col):
    low_limit, up_limit = catch_outliers(dataframe, num_col)

    dataframe.loc[(dataframe[num_col] < low_limit), num_col] = low_limit
    dataframe.loc[(dataframe[num_col] > up_limit), num_col] = up_limit

# Feature Extraction

## Binary Feature

In [5]:
df = load()

In [6]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


Titanic veriseti içerisinde Cabin değişkenine baktığımız zaman nan değerler görüyoruz. Bu değişken içerisinde bol miktarda boş veri bulunuyor. Bu verisetinde yeni bir değişken oluşturucağız. Bu değişken de cabin içerisinde nan olan değerler 1, dolu değerler 0 değerlerini alsın.

In [7]:
df["NEW_CABIN_BOOL"] = df["Cabin"].notnull().astype('int')

In [8]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_CABIN_BOOL
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,0


Yukarıdaki kod sayesinde, cabin değişkeni içerisinde dolu olan yerlere 1, boş olan yerlere 0 yazdık ve bunu bir değişkene atadık, sonrasında da değişkeni dataframe içerisine kaydettik.

Şimdi bu yeni oluşturduğumuz değişkenin, bağımlı değişkene göre oranını inceleyelim

In [9]:
df.groupby("NEW_CABIN_BOOL").agg({"Survived": "mean"})

,Survived
NEW_CABIN_BOOL,
0,0.300
1,0.667


Cabin numarası olanların %60'ı hayatta kalmış. Bu yüksek bir oran Dolayısıyla hiçbir anlam ifade etmeyen bir değişkenden bir anlam ifade eden değişken oluşturmuş olduk. 

Şimdi yeni oluşturduğumuz bir feature var, bu feature'ın istatistiki olarak bağımlı değişken ile arasındaki ilişkiyi merak ediyoruz. Bunun için oran testi yapacağız.

In [10]:
from statsmodels.stats.proportion import proportions_ztest

In [11]:
test_stat, pvalue = proportions_ztest(count=[df.loc[df["NEW_CABIN_BOOL"] == 1, "Survived"].sum(), #cabin nosu olan, hayatta kalan kaç kişi var?
                                            df.loc[df["NEW_CABIN_BOOL"] == 0, "Survived"].sum()], #cabin nosu olmayan, hayatta kalan kaç kişi var?
                                     
                                     nobs=[df.loc[df["NEW_CABIN_BOOL"] == 1, "Survived"].shape[0], #cabin nosu olanlar kaç kişi?
                                          df.loc[df["NEW_CABIN_BOOL"] == 0, "Survived"].shape[0]]) #cabin nosu olmayanlar kaç kişi?

In [12]:
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 9.4597, p-value = 0.0000


Proportion z testinin hipotezi, p1 ve p2 oranları arasında fark yoktur der. p1 ve p2 oranları, iki grubun, cabin numarası olanların ve olmayanların hayatta kalma oranları. İki arasında fark yoktur diyen hipotez, p-value değeri 0.05'ten küçük olduğundan dolayı reddedilir. Yani aralarında istatistiki olarak anlamlı bir farklılık var gibi gözüküyor.

Şimdi aynı şeyi başka bir değişken üzerinden yapalım. Veriseti içerisinde SibSp ve Parch değişkenleri bulunuyor. Bu değişkenler, o kişinin gemi içerisinde kaç tane yakını olduğunun bilgisini veren değişkenler. Yeni bir değişken oluşturalım ve bu iki değişkenin toplamına göre, kişinin o teknede yalnız olup olmadığının bilgisini versin.

In [13]:
df.loc[((df['SibSp'] + df['Parch']) > 0), "NEW_IS_ALONE"] = "NO"
df.loc[((df['SibSp'] + df['Parch']) == 0), "NEW_IS_ALONE"] = "YES"

In [14]:
df.groupby("NEW_IS_ALONE").agg({"Survived": "mean"})

,Survived
NEW_IS_ALONE,
NO,0.506
YES,0.304


Yukarıdaki veriye baktığımızda, gemide yalnız olmayanların, yani bir yakını bulunanların hayatta kalma oranları daha fazla gözüküyor. Yalnız olanların hayatta kalma oranları daha düşük.  

Yukarıda yaptığımız proportion testini şimdi bu değişkene de yapalım:

In [15]:
test_stat, pvalue = proportions_ztest(count=[df.loc[df["NEW_IS_ALONE"] == "YES", "Survived"].sum(), 
                                            df.loc[df["NEW_IS_ALONE"] == "NO", "Survived"].sum()], 
                                     
                                     nobs=[df.loc[df["NEW_IS_ALONE"] == "YES", "Survived"].shape[0],
                                          df.loc[df["NEW_IS_ALONE"] == "NO", "Survived"].shape[0]])

In [16]:
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -6.0704, p-value = 0.0000


p-value değerine bakıldığında h0 hipotezi reddedilir. h0 hipotezi der ki, iki oran arasında fark yoktur. 

## Text Features

Metinler üzerinden değişken üretmek

In [17]:
df = load()

In [18]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


Name değişkeni kardinal bir değişken gibi gözüküyor ama acaba içerisinden başka değişkenler çıkarabilir miyiz?

### Letter Count

Name değişkeni içerisindeki isimlerin harflerinin toplamını alırsak bir anlam çıkar mı?

In [19]:
df["NEW_NAME_COUNT"] = df["Name"].str.len()

In [20]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_NAME_COUNT
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,23
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,51
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,22
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,44
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,24


### Word Count

In [21]:
df["NEW_NAME_WORD_COUNT"] = df["Name"].apply(lambda x: len(str(x).split(" ")))

In [22]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_NAME_COUNT,NEW_NAME_WORD_COUNT
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,23,4
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,51,7
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,22,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,44,7
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,24,4


### Özel Yapıları Yakalamak

İsimlere bakıldığında kişilerin herhangi bir meslek bilgisi yok fakat bazı isimlerin önünde Dr. ifadesi yazıyor. Acaba burada doktor olanların 1 olmayanların 0 yazdığı bir değişken oluşturabilir miyiz?

In [24]:
df["NEW_NAME_DR"] = df["Name"].apply(lambda x: len([x for x in x.split() if x.startswith("Dr")]))

In [25]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_NAME_COUNT,NEW_NAME_WORD_COUNT,NEW_NAME_DR
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,23,4,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,51,7,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,22,3,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,44,7,0
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,24,4,0


In [28]:
df.groupby("NEW_NAME_DR").agg({"Survived": ["mean", "count"]})

Survived      
                mean count
NEW_NAME_DR               
0              0.383   881
1              0.500    10

Yaklaşık 900 gözlem birimi içerisinden sadece 10 kişinin doktor olması kayda değer bir anlam ifade eder mi bilemiyoruz, fakat yine de bir oran olduğundan dolayı bu bilgi bir kenarda dursun.

## Regex Features

Bu kısımda yine metinsel ifadeler üzerinde çalışacağız. İsimlerin önünde Mr., Mrs. ya da Miss gibi title'lar bulunuyor. Bunları çekip bir değişken oluşturabilir miyiz?

In [29]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_NAME_COUNT,NEW_NAME_WORD_COUNT,NEW_NAME_DR
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,23,4,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,51,7,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,22,3,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,44,7,0
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,24,4,0


In [32]:
# boşluk ile başlayıp nokta ile biten, ve büyük ve küçük harfler içerebilen ifadeleri yakala
df["NEW_TITLE"] = df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

In [33]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_NAME_COUNT,NEW_NAME_WORD_COUNT,NEW_NAME_DR,NEW_TITLE
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,23,4,0,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,51,7,0,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,22,3,0,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,44,7,0,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,24,4,0,Mr


In [35]:
df[["NEW_TITLE", "Survived", "Age"]].groupby(["NEW_TITLE"]).agg({"Survived": "mean",
                                                                "Age": ["count", "mean"]})

Survived   Age       
              mean count   mean
NEW_TITLE                      
Capt         0.000     1 70.000
Col          0.500     2 58.000
Countess     1.000     1 33.000
Don          0.000     1 40.000
Dr           0.429     6 42.000
Jonkheer     0.000     1 38.000
Lady         1.000     1 48.000
Major        0.500     2 48.500
Master       0.575    36  4.574
Miss         0.698   146 21.774
Mlle         1.000     2 24.000
Mme          1.000     1 24.000
Mr           0.157   398 32.368
Mrs          0.792   108 35.898
Ms           1.000     1 28.000
Rev          0.000     6 43.167
Sir          1.000     1 49.000

Burada kategorik değişken kırılımında değişken atama işlemini yapabilmek için bir yardımcı değişken oluşturmuş olduk. Normalde "Age" değişkeni içerisinde eksik değerler bulunuyor ve biz bunları genel olarak Age'in medyanı ile doldurabilirdik. Fakat burada görüyoruzki birçok title bulunuyor ve bunların hepsinin yaş ortalamaları farklı. Dolayısıyla her bir eksik yaş verisini kendi title'ının ortalama yaşı ile doldurursak daha anlamlı bir veriseti oluşturmuş oluruz.

## Date Features

In [37]:
dff =pd.read_csv("datasets/course_reviews.csv")

In [38]:
dff.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered
0,5.000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.000,0.000,0.000
1,5.000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.000,0.000,0.000
2,4.500,2021-02-04 20:34:03,2019-07-04 23:23:27,1.000,0.000,0.000
3,5.000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.000,0.000,0.000
4,4.000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.000,0.000,0.000


In [41]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rating              4323 non-null   float64
 1   Timestamp           4323 non-null   object 
 2   Enrolled            4323 non-null   object 
 3   Progress            4323 non-null   float64
 4   Questions Asked     4323 non-null   float64
 5   Questions Answered  4323 non-null   float64
dtypes: float64(4), object(2)
memory usage: 202.8+ KB


Amacımız dataframe içerisindeki timestamp değişkeninden yeni değişkenler üretmek fakat bu df'in infosuna bakarsak timestamp değişkeninin object tipinde olduğunu görürürüz. Öncelikle bu değişkenin tipini dönüştürmemiz gerekli.

In [42]:
# dönüştürmek istediğin değişkeni ver ve değişen içerisinde tarihler nasıl sıralanmış onun sırasını gir
dff["Timestamp"] = pd.to_datetime(dff["Timestamp"], format="%Y-%m-%d")

In [43]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Rating              4323 non-null   float64       
 1   Timestamp           4323 non-null   datetime64[ns]
 2   Enrolled            4323 non-null   object        
 3   Progress            4323 non-null   float64       
 4   Questions Asked     4323 non-null   float64       
 5   Questions Answered  4323 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 202.8+ KB


Şimdi buradan bir yıl değişkeni üretebiliriz.

In [44]:
# year
dff["year"] = dff["Timestamp"].dt.year # dt -> datetime modülü. defterin en başında import ettik

In [45]:
dff.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered,year
0,5.000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.000,0.000,0.000,2021
1,5.000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.000,0.000,0.000,2021
2,4.500,2021-02-04 20:34:03,2019-07-04 23:23:27,1.000,0.000,0.000,2021
3,5.000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.000,0.000,0.000,2021
4,4.000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.000,0.000,0.000,2021


In [47]:
# month
dff["month"] = dff["Timestamp"].dt.month 

In [48]:
dff.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered,year,month
0,5.000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.000,0.000,0.000,2021,2
1,5.000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.000,0.000,0.000,2021,2
2,4.500,2021-02-04 20:34:03,2019-07-04 23:23:27,1.000,0.000,0.000,2021,2
3,5.000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.000,0.000,0.000,2021,2
4,4.000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.000,0.000,0.000,2021,2


In [51]:
# year diff
dff["year_diff"] = date.today().year - dff['Timestamp'].dt.year
# günümüzdeki yıla göre kaç yıl oldu?

In [52]:
dff.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered,year,month,year_diff
0,5.000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.000,0.000,0.000,2021,2,1
1,5.000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.000,0.000,0.000,2021,2,1
2,4.500,2021-02-04 20:34:03,2019-07-04 23:23:27,1.000,0.000,0.000,2021,2,1
3,5.000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.000,0.000,0.000,2021,2,1
4,4.000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.000,0.000,0.000,2021,2,1


In [56]:
# month diff (iki tarih arasındaki ay farkı): yıl farkı + ay farkı
dff["month_df"] = (date.today().year - dff["Timestamp"].dt.year) * 12 + (date.today().month - dff["Timestamp"].dt.month)

In [57]:
dff.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered,year,month,year_diff,month_df
0,5.000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.000,0.000,0.000,2021,2,1,17
1,5.000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.000,0.000,0.000,2021,2,1,17
2,4.500,2021-02-04 20:34:03,2019-07-04 23:23:27,1.000,0.000,0.000,2021,2,1,17
3,5.000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.000,0.000,0.000,2021,2,1,17
4,4.000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.000,0.000,0.000,2021,2,1,17


In [58]:
# day_name
dff["day_name"] = dff['Timestamp'].dt.day_name()

In [59]:
dff.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered,year,month,year_diff,month_df,day_name
0,5.000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.000,0.000,0.000,2021,2,1,17,Friday
1,5.000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.000,0.000,0.000,2021,2,1,17,Thursday
2,4.500,2021-02-04 20:34:03,2019-07-04 23:23:27,1.000,0.000,0.000,2021,2,1,17,Thursday
3,5.000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.000,0.000,0.000,2021,2,1,17,Thursday
4,4.000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.000,0.000,0.000,2021,2,1,17,Thursday


## Feature Interaction (Özellik Etkileşimleri)

Feature Interaction, değişkenlerin birbirleri ile etkileşime girmesi demektir. Örneğin iki değişkenin çarpılması, iki değişkenin toplanması, bir değişkenin karesinin, küpünün alınması gibi değişkenlerle etkileşim kurmak demektir.

In [60]:
df = load()

In [61]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


Dataframe içerisinde değişkenleri birbirleri ile çeşitli işlemlere sokup çeşitli değişkenler üretebiliriz fakat bu interaction bir şey ifade ediyor olmalı 

In [62]:
df["NEW_AGE_PCLASS"] = df["Age"] * df["Pclass"]

Bu kodun arkasındaki temel mantık, yaşı büyük ya da küçük olanların yolculuk sınıflarına göre refah durumları ile ilgili bir durum ortaya çıkarmak. (yaşı küçük ama 1.sınıfta yolculuk yapıyor durumu iyi, yaşı büyük ama 3.sınıfta yolculuk, refah durumu düşük)

<b>Dikkat! Burada Age değişkeni bir standartlaştırmaya tabi tutulması gerekebilir.</b>

Mesela akrabalık sayıları ile ilgili iki değişkenin toplamı + kişinin kendisi dersek family size adında aile boyutu bilgisini veren bir değişken oluşturmuş oluruz.

In [63]:
df["NEW_FAMILY_SIZE"] = df["SibSp"] + df["Parch"] + 1

In [64]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_AGE_PCLASS,NEW_FAMILY_SIZE
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,66.000,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,38.000,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,78.000,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,35.000,2
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,105.000,1


Belirli kategorik değişken ya da sayısal değişkenlerin etkileşim noktalarına da flag atılabilir, değişken oluşturulabilir.

In [68]:
# erkek olup yaşı 21'den küçük veya eşit olanlar 'NEW_SEX_CAT' değişkeni altında youngmale olarak atanacak
df.loc[(df["Sex"] == 'male') & (df['Age'] <= 21), 'NEW_SEX_CAT'] = 'youngmale'

# erkek olup yaşı 21'den büyük ve 50'den küçük veya eşit olanlar maturemale
df.loc[(df['Sex'] == 'male') & ((df['Age'] > 21) & (df['Age'] <= 50)), 'NEW_SEX_CAT'] = 'maturemale'

# erkek olup yaşı 50'den büyük olanlar seniormale
df.loc[(df["Sex"] == 'male') & (df['Age'] > 50), 'NEW_SEX_CAT'] = 'seniormale'

In [69]:
# kadın olup yaşı 21'den küçük veya eşit olanlar 'NEW_SEX_CAT' değişkeni altında youngfemale olarak atanacak
df.loc[(df["Sex"] == 'female') & (df['Age'] <= 21), 'NEW_SEX_CAT'] = 'youngfemale'

# kadın olup yaşı 21'den büyük ve 50'den küçük veya eşit olanlar maturefemale
df.loc[(df['Sex'] == 'female') & ((df['Age'] > 21) & (df['Age'] <= 50)), 'NEW_SEX_CAT'] = 'maturefemale'

# kadın olup yaşı 50'den büyük olanlar seniorfemale
df.loc[(df["Sex"] == 'female') & (df['Age'] > 50), 'NEW_SEX_CAT'] = 'seniorfemale'

In [70]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_AGE_PCLASS,NEW_FAMILY_SIZE,NEW_SEX_CAT
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,66.000,2,maturemale
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,38.000,2,maturefemale
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,78.000,1,maturefemale
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,35.000,2,maturefemale
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,105.000,1,maturemale


Yeni oluşturduğumuz NEW_SEX_CAT değişkeni acaba bir anlam ifade ediyor mu?

In [74]:
df.groupby('NEW_SEX_CAT')['Survived'].mean().sort_values(ascending=False)

NEW_SEX_CAT
seniorfemale   0.941
maturefemale   0.775
youngfemale    0.679
youngmale      0.250
maturemale     0.199
seniormale     0.128
Name: Survived, dtype: float64

Yukarıdaki veriye göre beklendiği gibi yine kadınların hayatta kalma oranları daha fazla ama kadınlar da kendi içlerin de olgunluk durumuna göre hayatta kalma oranları değişmekte. Yine aynı şekilde erkeklerde de olgunluk durumuna göre hayatta kalma oranlarında farklılık var. Dolayısıyla bu değişken kullanılabilir duruyor.